In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

In [14]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [15]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [16]:
# Image proceesing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),
                        std=(0.5, 0.5, 0.5))
])


In [17]:
# MNIST dataset
mnist = datasets.MNIST(root='/home/damvantai/Documents/data/',
                      train=True,
                      transform=transform,
                      download=True)

In [18]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                         batch_size=100,
                                         shuffle=True)

In [19]:
data_loader

In [20]:
# Discriminator
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

In [21]:
# Generator
G = nn.Sequential(
    nn.Linear(64, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 784),
    nn.Tanh()
)

In [22]:
if torch.cuda.is_available():
    D.cuda()
    G.cuda()

In [23]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

In [26]:
# Start training
for epoch in range(200):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        batch_size = images.size(0)
        images = to_var(images.view(batch_size, -1))
        real_labels = to_var(torch.ones(batch_size))
        fake_labels = to_var(torch.ones(batch_size))
        
        # Train the discriminator
        
        # Compute loss with real images
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute loss with fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop + optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Train the generator
        
        # Conpute loss with fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i + 1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                 'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                 % (epoch, 200, i+1, 600, d_loss.data[0], g_loss.data[0],
                   real_score.data.mean(), fake_score.data.mean()))
    
    # Save real images
    if (epoch + 1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), './data/real_images.png')
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), './data/fake_image-%d.png' % (epoch+1))

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [0/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [0/200], Step[600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [1/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [1/200], Step[600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [2/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [2/200], Step[600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [3/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [3/200], Step[600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [4/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [4/200], Step[600/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [5/200], Step[300/600], d_loss: 0.0000, g_loss: 0.0000, D(x): 1.00, D(G(z)): 1.00
Epoch [5/200], Step[600/600], d_

KeyboardInterrupt: 